attentionベクトルをがんばってとりだすぞ

In [1]:
from keras.layers          import Lambda, Input, Dense, GRU, LSTM, Dropout
from keras.models          import Model
from keras.layers.wrappers import Bidirectional
from keras.callbacks       import LambdaCallback 
from keras.optimizers      import Adam
from keras.engine.topology import Layer
from keras.layers.normalization import BatchNormalization as BN
import keras.backend as K
import random
import sys
import pickle
import glob
import copy
import os
import re
import MeCab
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from gensim.models import word2vec
import numpy as np
from keras.utils import plot_model
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import initializers
model = word2vec.Word2Vec.load('/mnt/sdc/wikipedia_data/jawiki_wakati.model')

Using TensorFlow backend.


In [2]:
addDict={}
seq_len=13
categories=6

In [3]:
def predictVector(word, around_words_list):
    global addDict
    if word in addDict:
        return addDict[word]
    else:
        return addUnknownWord(word,around_words_list)

def addUnknownWord(word , around_words_list):
    global addDict
    rand_vector=np.random.rand(200)/np.linalg.norm(np.random.rand(200))*(10+ 3*np.random.rand(1))
    vector=np.array(model[model.predict_output_word(around_words_list)[0][0]])+rand_vector
    addDict[word]=vector
    return vector
    
def Wakati(text):
    m = MeCab.Tagger ("-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd -Owakati")
    result=m.parse(text)
    ws = re.compile(" ")
    words = [word for word in ws.split(result)]
    if words[-1] == u"\n":
        words = words[:-1]
    return [word for word in words if word!="「" and word!="」" and word!="、"]

def extractKeyword(text,word_class):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('') # <= 空文字列をparseする
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        if node.feature.split(",")[0] == word_class:
            keywords.append(node.surface)
        node = node.next
    return keywords

def seq2vecs(words,predict):
    global addDict
    vectors=[]
    for i in range(len(words)):
            try:
                vectors.append(model[words[i]])
            except:
                if predict:
                    try:
                        vectors.append(predictVector(words[i],[words[i-1]]))
                    except:
                        if i==0:
                            return []
                        else:
                            similar_word=model.similar_by_vector(addDict[words[i-1]], topn=10, restrict_vocab=None)[0][0]
                            vectors.append(predictVector(words[i],[similar_word]))
                else:
                    return []
    return vectors

In [4]:
def train_test_divide(X,Y,T,test_rate):
    datanum=len(X)
    n=math.floor(datanum*test_rate)
    X_train=np.array(X[:datanum-n])
    Y_train=np.array(Y[:datanum-n])
    X_test=np.array(X[datanum-n:])
    Y_test=np.array(Y[datanum-n:])
    T_train=np.array(T[:datanum-n])
    T_test=np.array(T[datanum-n:])
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    print(len(T_train))
    print(len(T_test))
    return (X_train,Y_train,T_train),(X_test,Y_test,T_test)

def onehot_vector(number):
    global categories
    onehot=np.zeros(categories)
    onehot[number]=1
    return onehot

In [5]:
def load_data(filename,sfl,predict,extract):
    global seq_len
    X=[]
    Y=[]
    T=[]
    sum=0
    max_len=0
    min_len=1000000
    with open(filename, "r") as f:
        lines = [line for line in f]
        for line in lines:
            title,category=line.split(" ")
            words=[]
            if extract:
                words=extractKeyword(title,"名詞")
            else:
                words=Wakati(title)
            input_vectors=seq2vecs(words,predict)
            sum+=len(input_vectors)
            max_len=max(max_len,len(input_vectors))
            if len(input_vectors) > seq_len:
                input_vectors=input_vectors[:seq_len]
            elif len(input_vectors)==0:
                continue
            min_len=min(min_len,len(input_vectors))
            if sfl:
                random.shuffle(input_vectors)
            x = [ [0.]*200 for _ in range(seq_len) ]
            x[0:len(input_vectors)]=input_vectors
            y=onehot_vector(int(category))
            X.append(np.array(x))
            Y.append(np.array(y))
            T.append(title)
    X,Y=shuffle(X,Y)
    X=np.array(X)
    Y=np.array(Y)
    print("-------DataShape------")
    print(X.shape)
    print(Y.shape)
    print(len(T))
    print("-------DataProperties------")
    print("file:"+filename)
    print("max:"+str(max_len))
    print("min:"+str(min_len))
    print("mean:"+str(sum/len(T)))
    return X,Y,T

In [6]:
class AttLayer(Layer):
    
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = K.variable(self.init((input_shape[-1],1)))
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x,self.W))
        eij=K.squeeze(eij,axis=2)
        ai = K.exp(eij)
        Sum=K.expand_dims(K.sum(ai, axis=1),axis=1)
        weights = ai/Sum
        weights=K.expand_dims(weights,axis=1)
        return weights

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])

In [7]:
X,Y,T=load_data("./data/yahoo_data.txt",sfl=False,predict=False,extract=False)

/home/hikaru/.pyenv/versions/anaconda3-4.4.0/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


-------DataShape------
(68258, 13, 200)
(68258, 6)
68258
-------DataProperties------
file:./data/yahoo_data.txt
max:13
min:2
mean:6.215784230419877


In [8]:
(X_train,Y_train,T_train),(X_test,Y_test,T_test)=train_test_divide(X,Y,T,0.1)

(61433, 13, 200)
(61433, 6)
(6825, 13, 200)
(6825, 6)
61433
6825


In [9]:
def encode(args):
    x,weights=args
    weighted_input = K.batch_dot(weights, x)
    weighted_input=K.squeeze(weighted_input,axis=1)
    return weighted_input

In [10]:
inputs      = Input(shape=(seq_len, 200))
x     = Bidirectional(LSTM(512,return_sequences=True,dropout=0.3))(inputs)
weights    = AttLayer(name="attention")(x)
encoded =Lambda(encode,output_shape=(1024,))([x,weights])
preds     = Dense(categories,activation='softmax',name="predict")(encoded)
LSTM_Model = Model(inputs=inputs, outputs=preds)
LSTM_Model.summary()
LSTM_Model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=["accuracy"])
history =LSTM_Model.fit(X_train,Y_train,shuffle=True,batch_size=30,epochs=1,
                   validation_split=0.3)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 13, 200)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 13, 1024)     2920448     input_1[0][0]                    
__________________________________________________________________________________________________
attention (AttLayer)            (None, 13)           1024        bidirectional_1[0][0]            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1024)         0           bidirectional_1[0][0]            
                                                                 attention[0][0]                  
__________

In [11]:
loss_and_metrics_yahoo = LSTM_Model.evaluate(X_test,Y_test)

6825/6825 [==============================] - 6s 824us/step


In [12]:
print("\n <Yahoo> loss:{} accuracy:{}"
      .format(loss_and_metrics_yahoo[0],loss_and_metrics_yahoo[1]))


 <Yahoo> loss:0.3581013316003394 accuracy:0.871941391941392


In [19]:
layer_name = "attention"
intermediate_layer_model = Model(inputs=LSTM_Model.input,
                                 outputs=LSTM_Model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_test)
attention_vectors=intermediate_output.reshape(intermediate_output.shape[0],intermediate_output.shape[-1])

In [24]:
for i in range(10):
    title=Wakati(T_test[i])+["EOS"]*(seq_len-len(Wakati(T_test[i])))
    attention_vector=attention_vectors[i]
    for j in range(seq_len):
        print(title[j]+":"+str(attention_vector[j]))
    print("----------------")

米:0.18819253
Google:0.19487444
地図:0.20320328
サービス:0.109083146
を:0.03740421
買収:0.033794228
EOS:0.033435278
EOS:0.03336167
EOS:0.033338692
EOS:0.033330247
EOS:0.033327617
EOS:0.033327468
EOS:0.033327155
----------------
Apple:0.28072456
物足りない:0.28085902
iOS:0.054863084
刷新:0.038112257
EOS:0.039688975
EOS:0.038744386
EOS:0.038281262
EOS:0.038166374
EOS:0.038127933
EOS:0.038113188
EOS:0.038107783
EOS:0.03810626
EOS:0.038104996
----------------
低迷:0.21629919
アップル:0.041298755
巻き返し:0.04121396
なるか:0.052997816
EOS:0.059528414
EOS:0.29434705
EOS:0.04298648
EOS:0.040136717
EOS:0.04956867
EOS:0.041521132
EOS:0.040211592
EOS:0.039975148
EOS:0.039915092
----------------
KDDI:0.20762764
通信:0.056700308
障害:0.03685419
客:0.02962886
に:0.2076427
700円:0.034572624
返金:0.20760392
EOS:0.07185859
EOS:0.032928754
EOS:0.029236063
EOS:0.028589869
EOS:0.028414888
EOS:0.0283417
----------------
米:0.23257907
Google:0.04153371
地図:0.22078328
サービス:0.031691145
買収:0.053943865
か:0.17693661
EOS:0.0490567
EOS:0.034412175
EOS:0